In [1]:
# Inputs: 
    # hybrid base zoning with allowed dev type and intensity at p10 parcel level
    # pba50zoningmod scenarios - dev type and intensity modifications at pba50zoningmodscat level
# Output:
    # pba50zoningmod dev type and intensity at parcel level
    # development capacity in res units, non-res sqft, and employment by parcel 

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [3]:
if os.getenv('USERNAME')    =='ywang':
    BOX_dir                 = 'C:\\Users\\{}\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim 1.5\\PBA50'.format(os.getenv('USERNAME'))
    GitHub_urbansim_dir     = 'C:\\Users\\{}\\Documents\\GitHub\\bayarea_urbansim'.format(os.getenv('USERNAME'))

    # input file locations
    hybrid_plu_boc_dir      = os.path.join(BOX_dir, 'Policies\\Base zoning\\outputs\\hybrid_base_zoning')
    pba50zoningmods_dir     = os.path.join(GitHub_urbansim_dir, 'data')
    other_inputs_dir        = os.path.join(BOX_dir, 'Policies\\Base zoning\\inputs')
    
    # output file location
    data_output_dir         = os.path.join(BOX_dir, 'Policies\\Zoning Modifications')


ALLOWED_BUILDING_TYPE_CODES = ["HS","HT","HM","OF","HO","SC","IL","IW","IH","RS","RB","MR","MT","ME"]
RES_BUILDING_TYPE_CODES     = ["HS","HT","HM",                                        "MR"          ]
NONRES_BUILDING_TYPE_CODES  = [               "OF","HO","SC","IL","IW","IH","RS","RB","MR","MT","ME"]
INTENSITY_CODES             = ['max_far','max_dua','max_height']

# used in impute_max_dua() and impute_max_far()
SQUARE_FEET_PER_ACRE                = 43560.0
SQUARE_FEET_PER_DU                  = 1200.0
FEET_PER_STORY                      = 11.0
PARCEL_USE_EFFICIENCY               = 0.5
SQUARE_FEET_PER_EMPLOYEE            = 350.0
SQUARE_FEET_PER_EMPLOYEE_OFFICE     = 175.0
SQUARE_FEET_PER_EMPLOYEE_INDUSTRIAL = 500.0

mods = '21'

today = datetime.today().strftime('%Y_%m_%d')

In [4]:
# Hybrid base zoning file
base_zoning_file = os.path.join(hybrid_plu_boc_dir, '2020_05_03_p10_plu_boc_BASIS_devType_intensity_partial.csv')
base_zoning = pd.read_csv(
    base_zoning_file,
    usecols = ['PARCEL_ID','county_id', 'county_name', 'juris_zmod','ACRES','pba50zoningmodcat_zmod','nodev_zmod',
              'max_far_basis','max_dua_basis'] + [
              devType+'_basis' for devType in ALLOWED_BUILDING_TYPE_CODES],
    dtype = {'nodev_zmod':np.int})

base_zoning.rename(columns = {
    'pba50zoningmodcat_zmod':'pba50zoningmodcat',
    'max_far_basis'         :'max_far',
    'max_dua_basis'         :'max_dua',
    'HS_basis'              :'HS',
    'HT_basis'              :'HT',
    'HM_basis'              :'HM',
    'OF_basis'              :'OF',
    'HO_basis'              :'HO',
    'SC_basis'              :'SC',
    'IL_basis'              :'IL',
    'IW_basis'              :'IW',
    'IH_basis'              :'IH',
    'RS_basis'              :'RS',
    'RB_basis'              :'RB',
    'MR_basis'              :'MR',
    'MT_basis'              :'MT',
    'ME_basis'              :'ME'},inplace = True)

In [5]:
base_zoning.head()

,PARCEL_ID,county_id,county_name,juris_zmod,ACRES,pba50zoningmodcat,max_far,max_dua,nodev_zmod,HS,...,HO,SC,IL,IW,IH,RS,RB,MR,MT,ME
0,229116.0,1,Alameda,livermore,3.360520,livermoreNANAHRADRNAinNA,NaN,2.00000,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,244166.0,1,Alameda,livermore,1.294423,livermoreNANADRNAinNA,NaN,3.00000,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,202378.0,1,Alameda,hayward,14.993605,haywardNANANANAinNA,1.363636,0.00000,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2004420.0,97,Sonoma,unincorporated_sonoma,316.247146,unincorporated_sonomaNANADRNAoutNA,1.590909,0.00417,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,340332.0,1,Alameda,fremont,0.621275,fremontNANAHRADRNAinNA,2.363636,23.00000,1,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
pba50zoningmods_file = os.path.join(pba50zoningmods_dir, 'zoning_mods_'+mods+'.csv')
pba50zoningmods = pd.read_csv(
    pba50zoningmods_file,
    usecols = ['pba50zoningmodcat','add_bldg', 'drop_bldg', 'dua_up', 'far_up', 'dua_down', 'far_down', 'res_rent_cat'])
pba50zoningmods.head()

,pba50zoningmodcat,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,res_rent_cat
0,alamedaGGNAHRANAinNA,HM,NaN,35.0,1.0,NaN,NaN,3.0
1,alamedaGGNAHRANAinsfd,HM,NaN,35.0,1.0,NaN,NaN,3.0
2,alamedaGGNAHRADRNAinNA,HM,NaN,35.0,1.0,NaN,NaN,3.0
3,alamedaGGNAHRADRNAinsfd,HM,NaN,35.0,1.0,NaN,NaN,3.0
4,alamedaGGNANANAinNA,HM,NaN,35.0,1.0,NaN,NaN,3.0


In [7]:
## Merge base zoning with pba50 zoning scenario
base_zoning_pba50 = base_zoning.merge(pba50zoningmods, on = 'pba50zoningmodcat', how = 'left')

In [8]:
## Get a list of development types that have modifications in pba50zoningmod

add_bldg_types = list(pba50zoningmods.add_bldg.dropna().unique())
print(add_bldg_types)
drop_bldg_types = list(pba50zoningmods.drop_bldg.dropna().unique())
print(drop_bldg_types)

['HM', 'IW', 'HS']
[]


In [9]:
## Make a copy to avoid modifying the base zoning
zoning_pba50_type = base_zoning_pba50.copy()

## Modify allowed dev type
if len(add_bldg_types) > 0:
    for type in add_bldg_types:
        add_bldg_parcels = zoning_pba50_type.add_bldg.notnull()
        zoning_pba50_type.loc[add_bldg_parcels,type] = 1

if len(drop_bldg_types) > 0:
    for type in drop_bldg_types:
        drop_bldg_parcels = zoning_pba50_type.drop_bldg.notnull()
        zoning_pba50_type.loc[drop_bldg_parcels,type] = 0

In [10]:
# Compare allowed dev types before and after applying pba50zoningmod
for devType in add_bldg_types + drop_bldg_types:
    print('Before applying pba50zoningmod dev type adjustment: out of {:,} parcels, {:,} parcels allowing {}'.format(
              base_zoning_pba50.shape[0], base_zoning_pba50.loc[base_zoning_pba50[devType] == 1].shape[0], devType), '\n',
          'After applying pab50zoningmod dev type adjustment: out of {:,} parcels, {:,} parcels allowing {}'.format(
              zoning_pba50_type.shape[0], zoning_pba50_type.loc[zoning_pba50_type[devType] == 1].shape[0], devType))

Before applying pba50zoningmod dev type adjustment: out of 1,956,208 parcels, 456,405 parcels allowing HM 
 After applying pab50zoningmod dev type adjustment: out of 1,956,208 parcels, 785,717 parcels allowing HM
Before applying pba50zoningmod dev type adjustment: out of 1,956,208 parcels, 42,302 parcels allowing IW 
 After applying pab50zoningmod dev type adjustment: out of 1,956,208 parcels, 554,473 parcels allowing IW
Before applying pba50zoningmod dev type adjustment: out of 1,956,208 parcels, 1,513,117 parcels allowing HS 
 After applying pab50zoningmod dev type adjustment: out of 1,956,208 parcels, 1,720,921 parcels allowing HS


In [11]:
## make a copy to separate intensity modifications from dev type modifications
zoning_pba50_type_intensity = zoning_pba50_type.copy()

## modify max_dua when 'dua_up' is available
dua_up_parcels = zoning_pba50_type_intensity.dua_up.notnull()
## the effective max_dua is the larger of base zoning max_dua and the pba50 max_dua
zoning_pba50_type_intensity.loc[dua_up_parcels,'max_dua'] = zoning_pba50_type_intensity[['max_dua', 'dua_up']].max(axis=1)

dua_down_parcels = zoning_pba50_type_intensity.dua_down.notnull()
zoning_pba50_type_intensity.loc[dua_down_parcels,'max_dua'] = zoning_pba50_type_intensity[['max_dua', 'dua_down']].min(axis=1)

far_up_parcels = zoning_pba50_type_intensity.far_up.notnull()
zoning_pba50_type_intensity.loc[far_up_parcels,'max_far'] = zoning_pba50_type_intensity[['max_far', 'far_up']].max(axis=1)

far_down_parcels = zoning_pba50_type_intensity.far_down.notnull()
zoning_pba50_type_intensity.loc[far_down_parcels,'max_far'] = zoning_pba50_type_intensity[['max_far', 'far_down']].min(axis=1)


## Compare max_dua and max_far before and after applying pba50zoningmod
print('Before applying pba50zoningmod intensity adjustment: \n', zoning_pba50_type[['max_dua','max_far']].describe())
print('After applying pba50zoningmod intensity adjustment: \n', zoning_pba50_type_intensity[['max_dua','max_far']].describe())

Before applying pba50zoningmod intensity adjustment: 
             max_dua       max_far
count  1.863772e+06  1.446051e+06
mean   1.524419e+01  1.137449e+00
std    3.134172e+01  7.120975e+00
min    0.000000e+00  0.000000e+00
25%    6.000000e+00  6.000000e-01
50%    8.700000e+00  1.090909e+00
75%    1.400000e+01  1.590909e+00
max    9.075000e+02  7.200000e+03
After applying pba50zoningmod intensity adjustment: 
             max_dua       max_far
count  1.905564e+06  1.563457e+06
mean   2.090533e+01  1.236068e+00
std    3.527853e+01  6.877226e+00
min    0.000000e+00  0.000000e+00
25%    6.000000e+00  4.500000e-01
50%    9.000000e+00  1.136364e+00
75%    3.000000e+01  1.800000e+00
max    9.075000e+02  7.200000e+03


In [12]:
# Assign allow residential and/or non-residential by summing the columns
# for the residential/nonresidential allowed building type codes
# Returns dataframe with PARCEL_ID, allow_res_[boc_source], allow_nonres_[boc_source]
def set_allow_dev_type(df_original):
    # don't modify passed df
    df = df_original.copy()

    # note that they can't be null because then they won't sum -- so make a copy and fillna with 0
    for dev_type in ALLOWED_BUILDING_TYPE_CODES:
        df[dev_type] = df[dev_type].fillna(value=0.0)    
    
    # allow_res is sum of allowed building types that are residential
    res_allowed_columns = RES_BUILDING_TYPE_CODES
    df['allow_res'] = df[res_allowed_columns].sum(axis=1)
    
    # allow_nonres is the sum of allowed building types that are non-residential
    nonres_allowed_columns = NONRES_BUILDING_TYPE_CODES
    df['allow_nonres'] = df[nonres_allowed_columns].sum(axis=1)
    
    return df[['PARCEL_ID',
               "allow_res",
               "allow_nonres"]]

In [13]:
# Calculate capacity

def calculate_capacity(df):
    
    # DUA calculations apply to parcels 'allowRes' and not marked as "nodev"
    df['units'] = df['ACRES'] * df['max_dua']   
    
    # zero out units for 'nodev' parcels or parcels that don't allow residential
    zero_unit_idx = (df['allow_res'] == 0) | (df['nodev_zmod'] == 1)
    df.loc[zero_unit_idx,'units'] = 0
        
    # FAR calculations apply to parcels 'allowNonRes' and not marked as "nodev"
    df['sqft'] = df['ACRES'] * df['max_far'] * SQUARE_FEET_PER_ACRE 
    
    # zero out sqft for 'nodev' parcels or parcels that don't allow non-residential
    zero_sqft_idx = (df['allow_nonres'] == 0) | (df['nodev_zmod'] == 1)
    df.loc[zero_sqft_idx,'sqft '      ] = 0
    
    df['Ksqft'] = df['sqft']*0.001

    # of nonresidential uses, only office allowed
    office_idx   = (df['OF'] == 1) & (df['allow_nonres']== 1)
    # of nonresidential uses, only industrial allowed
    allow_indust = df[['IL','IW','IH']].sum(axis = 1)
    indust_idx   = (allow_indust > 0) & (df['allow_nonres'] == allow_indust)
    # calculate non-residential capacity in employment
    df[               'emp'] = df['sqft'] / SQUARE_FEET_PER_EMPLOYEE
    df.loc[office_idx,'emp'] = df['sqft'] / SQUARE_FEET_PER_EMPLOYEE_OFFICE
    df.loc[indust_idx,'emp'] = df['sqft'] / SQUARE_FEET_PER_EMPLOYEE_INDUSTRIAL
    

    return df[['PARCEL_ID', 'ACRES', 'nodev_zmod', 'max_dua', 'max_far', 
                'allow_res', 'units', 'allow_nonres', 'sqft', 'Ksqft','emp']]

In [14]:
## Get pba50zoningmod allowed_res and allowed_nonres

pba50zoningmod_allowed_dev_type = set_allow_dev_type(zoning_pba50_type_intensity)
pba50_devType = zoning_pba50_type_intensity.merge(pba50zoningmod_allowed_dev_type, how = 'left', on = 'PARCEL_ID')

In [15]:
## Calculate pba50zoningmod development capacity

pab50_capacity = calculate_capacity(pba50_devType)

capacity_pba50_allAtts = pab50_capacity.merge(
    base_zoning_pba50[['PARCEL_ID','county_id', 'county_name', 'juris_zmod','pba50zoningmodcat']],
    on = 'PARCEL_ID', how = 'left')

In [16]:
for i in ['PARCEL_ID', 'nodev_zmod','allow_res','allow_nonres']:
    capacity_pba50_allAtts[i] = capacity_pba50_allAtts[i].astype('int64')

In [17]:
capacity_pba50_allAtts.to_csv(os.path.join(data_output_dir, today+'_capacity_pba50_allAtts.csv'), index = False)